# Chapter 2: Deep Learning and Language: The Basics

In [ ]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.layers.core import Dense, Activation, Dropout
# ... process data
model = Sequential()
model.add(Dense(128, input_dim=input_dim))
model.add(Activation('relu'))

## 2.1: Basic Architectures of Deep Learning

### 2.1.1: Deep Multilayer Perceptrons

### 2.1.2: Spatial and Temporal Operators

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
import pandas as pd
import sys
from keras.utils.vis_utils import plot_model


Using TensorFlow backend.


In [6]:
data = pd.read_csv('./raaijmakers-master-code/code/pos_neg.tsv.100', sep='\t') # tsv file
max_words = 1000
tokenizer = Tokenizer(num_words=max_words, split=' ')

tokenizer.fit_on_texts(data['text'].values)

X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(data['label']).values

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 36)



UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 29: invalid continuation byte

## 2.2 Deep Learning and NLP: A new paradigm